# Weavel

>[Weavel](https://weavel.ai) is product analytics for conversational AI.  
>Once integrated into your product, reports will be automatically generated and updated regularly based on conversation traces.   
>
>Check out [Weavel docs](https://weavel.ai/docs) for more information on **Weavel**.  

In this guide, we will go over how to set up the `WeavelCallbackHandler`. 


## Installation and Setup

In [ ]:
%pip install --upgrade --quiet langchain langchain-community

### Getting Weavel Credentials

If you do not have a Weavel account, create one on [here](https://analytics.weavel.ai/) and get your API key from the **Settings > Developers** section.

Check out our [Integration Guide](https://weavel.ai/docs/sdks/python) for more information.

Now set your API key as an environment variable:

In [ ]:
import os

os.environ["WEAVEL_API_KEY"] = "YoUr_ApI_KeY"  # replace with your API key

In [ ]:
from langchain_community.callbacks.weavel_callback import WeavelCallbackHandler

### Usage

`WeavelCallbackHandler` receives 2 required arguments (`user_id`, `trace_id`) and 4 optional arguments.  

Make sure to keep track of user and trace IDs in your system to keep the conversation trace consistent. 


```python
class WeavelCallbackHandler(BaseCallbackHandler):
    """
    Callback handler for Weavel.

    Args:
        user_id (str): User ID (Unique identifier for the user.)
        trace_id (str): Trace(Conversation) ID. (Unique identifier for the conversation.)
        user_message_id (str, Optional): Unique identifier for the user message, used to attach metadata to the user message.
        user_message_metadata (Dict[str, Any], Optional): Metadata to attach to the user message.
        assistant_message_id (str, Optional): Unique identifier for the assistant message, used to attach metadata to the assistant message.
        assistant_message_metadata (Dict[str, Any], Optional): Metadata to attach to the assistant message.
    """
```

## Usage

### Weavel callback within a chat model

Use `WeavelCallbackHandler` to log user and assistant messages automatically.

In [ ]:
os.environ["OPENAI_API_KEY"] = "sk-***"

In [ ]:
from uuid import uuid4

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

In [ ]:
chat_llm = ChatOpenAI()

user_id = str(uuid4())
trace_id = str(uuid4())

callbacks = [
    WeavelCallbackHandler(
        user_id=user_id,
        trace_id=trace_id,
    )
]

config = {
    "callbacks": callbacks,
}

In [ ]:
chat_res = chat_llm.invoke(
    [
        SystemMessage(content="You are a helpful assistant"),
        HumanMessage(content="I love you!"),
    ],
    config=config,
)

### Weavel callback within Chains 

`WeavelCallbackHandler` can be used with the Langchain `Chain` object.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [("system", "You are a helpful assistant."), ("user", "{input}")]
)

chat_llm = ChatOpenAI()

chain = prompt | chat_llm

callbacks = [
    WeavelCallbackHandler(
        user_id=user_id,
        trace_id=trace_id,
    )
]

config = {
    "callbacks": callbacks,
}

chat_res = chain.invoke(
    [
        "I love you!",
    ],
    config=config,
)

### Weavel callback within Agents

`WeavelCallbackHandler` can also be used with the Langchain `AgentExecutor` object.
When used within AgentExecutor, tool usage of the agent will be logged to Weavel as well.

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.tools import tool


@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' times 'y'."""
    print(f"Multiplying {x} by {y}")
    return x * y


@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the 'y'."""
    print(f"Raising {x} to the power of {y}")
    return x**y


@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    print(f"Adding {x} and {y}")
    return x + y


tools = [multiply, exponentiate, add]

agent_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "you're a helpful assistant"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

callbacks = [
    WeavelCallbackHandler(
        user_id=user_id,
        trace_id=trace_id,
    )
]

config = {
    "callbacks": callbacks,
}

chat_llm = ChatOpenAI()

agent = create_tool_calling_agent(chat_llm, tools, agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


agent_executor.invoke(
    {
        "input": "what's 3 plus 5 raised to the 2.743. also what's 17.24 - 918.1241",
    },
    config=config,
)

## More logging

You can record any user events such as button clicks or page views to Weavel through our [python SDK](https://weavel.ai/docs/sdks/python) or by sending a HTTP POST request to `https://api.weavel.ai/capture/track_event` ([REST API docs](https://api.weavel.ai/docs))

### Installing Weavel python SDK

In [ ]:
%pip install weavel

In [ ]:
import os

import requests
from weavel import create_client

WEAVEL_API_KEY = os.environ["WEAVEL_API_KEY"]

client = create_client()

client.track(
    user_id=user_id,
    trace_id=trace_id,  # trace_id is optional
    event_name="paid",
    properties={"amount": "100"},
)

client.close()
res = requests.post(
    "https://api.weavel.ai/capture/track_event",
    headers={"Authorization": f"Bearer {WEAVEL_API_KEY}"},
    json={
        "user_id": user_id,
        "trace_id": trace_id,  # trace_id is optional
        "name": "subscription",
        "properties": {"plan": "pro"},
    },
)